In [17]:
import imageio
import os
import json

dds_folder = r"0ad-textures-1"
output_image_folder = "PNG"
output_metadata_folder = "Metadata"

os.makedirs(output_image_folder, exist_ok=True)
os.makedirs(output_metadata_folder, exist_ok=True)

def extract_dds_metadata(dds_path):
    metadata = {}
    with open(dds_path, "rb") as f:
        header = f.read(128)  # Standard DDS header
        if header[0:4] != b'DDS ':
            raise ValueError("Invalid DDS magic number")

        # Extract FourCC (compression format) from the DDS_PIXELFORMAT (at offset 84)
        fourcc = header[84:88]
        fourcc_str = fourcc.decode("ascii", errors="ignore")
        metadata["magic"] = "DDS "
        metadata["header_hex"] = header.hex()
        metadata["fourcc"] = fourcc_str

        # Check for DX10 extended header
        if fourcc_str == "DX10":
            dx10_header = f.read(20)
            metadata["dx10_header_hex"] = dx10_header.hex()
            metadata["has_dx10_header"] = True
        else:
            metadata["has_dx10_header"] = False

    return metadata

for filename in os.listdir(dds_folder):
    if filename.lower().endswith(".dds"):
        dds_path = os.path.join(dds_folder, filename)

        # Load image
        try:
            image = imageio.imread(dds_path, format='dds')
        except Exception as e:
            print(f"[!] Failed to load DDS image: {filename} → {e}")
            continue

        # Save as PNG
        png_filename = os.path.splitext(filename)[0] + ".png"
        png_path = os.path.join(output_image_folder, png_filename)
        imageio.imwrite(png_path, image, format='png')

        # Extract and save metadata
        try:
            metadata = extract_dds_metadata(dds_path)
            meta_filename = os.path.splitext(filename)[0] + ".json"
            meta_path = os.path.join(output_metadata_folder, meta_filename)
            with open(meta_path, "w") as f:
                json.dump(metadata, f, indent=2)
            print(f"[✓] {filename} → {png_filename} + metadata ({'DX10' if metadata['has_dx10_header'] else 'std'})")
        except Exception as e:
            print(f"[!] Failed to extract metadata: {filename} → {e}")


C:\Users\hp\AppData\Local\Temp\ipykernel_17560\1244544625.py:42: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(dds_path, format='dds')


[✓] alpine_cliff.dds → alpine_cliff.png + metadata (std)
[✓] alpine_cliff_a.dds → alpine_cliff_a.png + metadata (std)
[✓] alpine_cliff_b.dds → alpine_cliff_b.png + metadata (std)
[✓] alpine_cliff_c.dds → alpine_cliff_c.png + metadata (std)
[✓] alpine_cliff_snow.dds → alpine_cliff_snow.png + metadata (std)
[✓] alpine_dirt.dds → alpine_dirt.png + metadata (std)
[✓] alpine_dirt_grass_50.dds → alpine_dirt_grass_50.png + metadata (std)
[✓] alpine_dirt_snow.dds → alpine_dirt_snow.png + metadata (std)
[✓] alpine_forrestfloor.dds → alpine_forrestfloor.png + metadata (std)
[✓] alpine_forrestfloor_snow.dds → alpine_forrestfloor_snow.png + metadata (std)
[✓] alpine_grass_rocky.dds → alpine_grass_rocky.png + metadata (std)
[✓] alpine_grass_snow_50.dds → alpine_grass_snow_50.png + metadata (std)
[✓] alpine_mountainside.dds → alpine_mountainside.png + metadata (std)
[✓] alpine_shore_rocks.dds → alpine_shore_rocks.png + metadata (std)
[✓] alpine_shore_rocks_grass_50.dds → alpine_shore_rocks_grass_50.

In [2]:
import vpk
import os

vpk_path = 'VPK/tf2_misc_dir.vpk'
output_dir = 'ExtractedVPK/'

with vpk.open(vpk_path) as archive:
    for file_path in archive:
        if file_path.endswith('.vtf'):
            print("Extracting:", file_path)
            file_data = archive.read_file(file_path)
            
            file_name = os.path.basename(file_path)
            output_path = os.path.join(output_dir, file_name)
            with open(output_path, 'wb') as f:
                f.write(file_data)

print("✅ Extracted all .vtf files.")

✅ Extracted all .vtf files.


In [8]:
import os
import subprocess

vtf_folder = r"ExtractedVPK\\"
png_output = r"PNG"
vtfcmd_path = r"vtflib132-bin/bin/x64/VTFCmd.exe" 

# Make sure the output folder exists
os.makedirs(png_output, exist_ok=True)

# Loop through and convert
for filename in os.listdir(vtf_folder):
    if filename.lower().endswith(".vtf"):
        input_path = os.path.join(vtf_folder, filename)
        print("🔄 Converting:", input_path)
        subprocess.run([
            vtfcmd_path,
            "-file", input_path,
            "-output", png_output,
            "-exportformat", "png"
        ])


🔄 Converting: ExtractedVPK\\donev2.vtf


In [18]:
pip install vtf2img

  Obtaining dependency information for vtf2img from https://files.pythonhosted.org/packages/78/04/8248534c3b357e0aaceae5b7728ff49e2244bfd8858962cc53ad77716433/vtf2img-0.1.0-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import subprocess
import re
import json
import os

vtfcmd_path = r"vtflib132-bin/bin/x64/VTFCmd.exe"
vtf_folder = r"ExtractedVPK"
json_output = r"Metadata"
png_output = "PNG"


patterns = {
    "version": r"Version:\s*v(\d+\.\d+)",
    "width": r"Width:\s*(\d+)",
    "height": r"Height:\s*(\d+)",
    "depth": r"Depth:\s*(\d+)",
    "frames": r"Frames:\s*(\d+)",
    "start_frame": r"Start Frame:\s*(\d+)",
    "faces": r"Faces:\s*(\d+)",
    "mipmaps": r"Mipmaps:\s*(\d+)",
    "flags": r"Flags:\s*(0x[0-9A-Fa-f]+)",
    "format": r"Format:\s*([^\n\r]+)",
    "reflectivity": r"Reflectivity:\s*([\d\.]+),\s*([\d\.]+),\s*([\d\.]+)"
}

for filename in os.listdir(vtf_folder):
    vtf_file = os.path.join(vtf_folder, filename)
    result = subprocess.run(
        [vtfcmd_path, "-file", vtf_file , "-output", png_output ,"-exportformat","png"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )

    output = result.stdout
    print(output) 

    metadata = {}

    for key, pattern in patterns.items():
        match = re.search(pattern, output)
        if match:
            if key == "reflectivity":
                metadata[key] = {
                    "r": float(match.group(1)),
                    "g": float(match.group(2)),
                    "b": float(match.group(3))
                }
            elif key in ["width", "height", "depth", "frames", "start_frame", "faces", "mipmaps"]:
                metadata[key] = int(match.group(1))
            elif key == "version":
                metadata[key] = float(match.group(1))
            else:
                metadata[key] = match.group(1).strip()

    json_file_path = os.path.join(json_output, filename + ".json")
    
    with open(json_file_path, "w") as f:
        json.dump(metadata, f, indent=4)

    print("Metadata saved to", json_output)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'ExtractedVPK'

In [15]:
import os
import subprocess
import json

vtfcmd_path = r"vtflib132-bin/bin/x64/VTFCmd.exe"
png_folder = "PNG"
metadata_folder = "Metadata"
output_folder = "ReconstructedVTF"

os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(png_folder):
    if not filename.lower().endswith(".png"):
        continue

    name = os.path.splitext(filename)[0]
    if name.endswith(".vtf") or name.endswith(".dds"):
        # Remove .vtf/.dds suffix from name
        name = os.path.splitext(name)[0]

    png_path = os.path.join(png_folder, filename)

    # Try both .json and .vtf.json metadata filenames
    possible_meta_files = [
        f"{name}.json",
        f"{name}.vtf.json",
        f"{name}.dds.json"
    ]

    json_path = None
    for meta_filename in possible_meta_files:
        meta_candidate = os.path.join(metadata_folder, meta_filename)
        if os.path.exists(meta_candidate):
            json_path = meta_candidate
            break

    if not json_path:
        print(f"[!] Metadata not found for {filename}")
        continue

    with open(json_path, "r") as f:
        metadata = json.load(f)

    output_path = os.path.join(output_folder, f"{name}.vtf")

    vtf_format = metadata.get("format", "DXT1")
    mipmaps = str(metadata.get("mipmaps", 1))

    command = [
        vtfcmd_path,
        "-file", png_path,
        "-output", output_folder,
        "-format", vtf_format,
        "-mipmaps", mipmaps
    ]

    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    if result.returncode == 0:
        print(f"[✓] Reconstructed {name}.vtf")
    else:
        print(f"[!] Error reconstructing {name}.vtf:")
        if result.stderr.strip():
            print(result.stderr.strip())
        else:
            print("(No stderr output, check if VTFCmd.exe is running correctly)")


[!] Error reconstructing alpine_cliff.vtf:
(No stderr output, check if VTFCmd.exe is running correctly)
[!] Error reconstructing alpine_cliff.vtf:
(No stderr output, check if VTFCmd.exe is running correctly)
[!] Error reconstructing alpine_cliff_a.vtf:
(No stderr output, check if VTFCmd.exe is running correctly)
[!] Error reconstructing alpine_cliff_a.vtf:
(No stderr output, check if VTFCmd.exe is running correctly)
[!] Error reconstructing alpine_cliff_b.vtf:
(No stderr output, check if VTFCmd.exe is running correctly)
[!] Error reconstructing alpine_cliff_b.vtf:
(No stderr output, check if VTFCmd.exe is running correctly)
[!] Error reconstructing alpine_cliff_c.vtf:
(No stderr output, check if VTFCmd.exe is running correctly)
[!] Error reconstructing alpine_cliff_c.vtf:
(No stderr output, check if VTFCmd.exe is running correctly)
[!] Error reconstructing alpine_cliff_snow.vtf:
(No stderr output, check if VTFCmd.exe is running correctly)
[!] Error reconstructing alpine_cliff_snow.vtf:

In [13]:
import os
import json
import imageio.v3 as iio
import numpy as np
import struct

png_folder = "PNG"
metadata_folder = "Metadata"
output_dds_folder = "ReconstructedDDS"

os.makedirs(output_dds_folder, exist_ok=True)

def create_dds_header(width, height, fourcc):
    dwSize = 124
    dwFlags = 0x0002100F  # CAPS | HEIGHT | WIDTH | PIXELFORMAT | LINEARSIZE
    dwPitchOrLinearSize = width * 4  # Assuming RGBA8
    dwCaps = 0x1000  # CAPS_TEXTURE
    dwCaps2 = 0

    pfSize = 32
    pfFlags = 0x41  # DDPF_RGB | DDPF_ALPHAPIXELS
    pfFourCC = fourcc.encode('ascii')
    pfRGBBitCount = 32
    pfRBitMask = 0x00FF0000
    pfGBitMask = 0x0000FF00
    pfBBitMask = 0x000000FF
    pfABitMask = 0xFF000000

    header = b"DDS "
    header += struct.pack("<I", dwSize)
    header += struct.pack("<I", dwFlags)
    header += struct.pack("<I", height)
    header += struct.pack("<I", width)
    header += struct.pack("<I", dwPitchOrLinearSize)
    header += struct.pack("<I", 0)  # dwDepth
    header += struct.pack("<I", 0)  # dwMipMapCount
    header += b"\x00" * 44  # Reserved1

    # DDS_PIXELFORMAT (32 bytes)
    header += struct.pack("<I", pfSize)
    header += struct.pack("<I", pfFlags)
    header += pfFourCC.ljust(4, b'\x00')  # pad FourCC to 4 bytes
    header += struct.pack("<I", pfRGBBitCount)
    header += struct.pack("<I", pfRBitMask)
    header += struct.pack("<I", pfGBitMask)
    header += struct.pack("<I", pfBBitMask)
    header += struct.pack("<I", pfABitMask)

    # Caps
    header += struct.pack("<I", dwCaps)
    header += struct.pack("<I", dwCaps2)
    header += b"\x00" * 12  # Caps3, Caps4, Reserved2

    return header

def clean_name(filename):
    name = filename
    if name.endswith(".dds.png"):
        name = name[:-8]  # Remove .dds.png
    elif name.endswith(".png"):
        name = name[:-4]  # Remove .png
    return name

for filename in os.listdir(png_folder):
    if filename.lower().endswith(".png"):
        base_name = clean_name(filename)
        png_path = os.path.join(png_folder, filename)
        json_path = os.path.join(metadata_folder, base_name + ".json")

        if not os.path.exists(json_path):
            print(f"[!] Missing metadata for {filename}")
            print(f"    → Tried: {json_path}")
            continue

        with open(json_path, "r") as f:
            metadata = json.load(f)

        fourcc = metadata.get("fourcc", "DXT1")[:4]
        image = iio.imread(png_path)
        height, width = image.shape[:2]

        if image.shape[2] == 3:
            image = np.dstack((image, np.full((height, width), 255, dtype=np.uint8)))

        rgba_bytes = image.astype(np.uint8).tobytes()
        dds_header = create_dds_header(width, height, fourcc)
        dds_data = dds_header + rgba_bytes

        dds_path = os.path.join(output_dds_folder, base_name + ".dds")
        with open(dds_path, "wb") as f:
            f.write(dds_data)

        print(f"[✓] Reconstructed {base_name}.dds from {filename} + metadata")


[✓] Reconstructed alpine_cliff.dds from alpine_cliff.dds.png + metadata
[✓] Reconstructed alpine_cliff.dds from alpine_cliff.png + metadata
[✓] Reconstructed alpine_cliff_a.dds from alpine_cliff_a.dds.png + metadata
[✓] Reconstructed alpine_cliff_a.dds from alpine_cliff_a.png + metadata
[✓] Reconstructed alpine_cliff_b.dds from alpine_cliff_b.dds.png + metadata
[✓] Reconstructed alpine_cliff_b.dds from alpine_cliff_b.png + metadata
[✓] Reconstructed alpine_cliff_c.dds from alpine_cliff_c.dds.png + metadata
[✓] Reconstructed alpine_cliff_c.dds from alpine_cliff_c.png + metadata
[✓] Reconstructed alpine_cliff_snow.dds from alpine_cliff_snow.dds.png + metadata
[✓] Reconstructed alpine_cliff_snow.dds from alpine_cliff_snow.png + metadata
[✓] Reconstructed alpine_dirt.dds from alpine_dirt.dds.png + metadata
[✓] Reconstructed alpine_dirt.dds from alpine_dirt.png + metadata
[✓] Reconstructed alpine_dirt_grass_50.dds from alpine_dirt_grass_50.dds.png + metadata
[✓] Reconstructed alpine_dirt_gr